## V1 PUBLIC

## Scraping Web with Numpy and Cv2

In [ ]:
import urllib3
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
import datetime
import time
import os
import requests
import random
import cv2
import numpy as np


def LerArquivo(subdiretorio):
    x = datetime.datetime.now()
    data = x.strftime("%Y%m%d")
    arq = open(subdiretorio,'r', encoding='utf-8')
    #print(arq.read())
    lista = []
    for item in arq:
        #print(item)
        lista.append(str(item).strip())
    print(lista)
    arq.close()
    return lista
    
def gravarNoArquivoUrl(n1,n2,diretorio):
    x = datetime.datetime.now()
    data = x.strftime('%d-%m-%Y_%H')
    arquivo = open(diretorio+'\\catalogo_resultado_'+ data+'.csv', 'a',encoding='utf-8')
    arquivo.write('%s;%s\n' %(n1,n2))
    arquivo.flush()
    arquivo.close()
    
def gerarImagensConcatenadas():
    #ref(tks) https://note.nkmk.me/en/python-opencv-hconcat-vconcat-np-tile/
    path = '.\Jornais'
    diretorio = os.listdir(path)
    for a in diretorio:
        pathSub = path + '\\'+ a
        #subdiretorio =  os.listdir(pathSub)
        for b in os.listdir(pathSub):
            if 'Images' in b:
                subdiretorio2 = pathSub + '\\' + b
                #print(subdiretorio2)
                list_array = []
                list_array_temp = []
                contador = 0
                for images in os.listdir(subdiretorio2):
                    print(pathSub)
                    image = subdiretorio2.replace('\\','//', 3) +'/'+ images
                    #print(subdiretorio2)
                    #ler imagens
                    imm1 = cv2.imread(image)
                    #deixar imagens no mesmo padrao
                    imm2 = cv2.resize(imm1, (1280, 720))
                    #Entender
                    im1_s = cv2.resize(imm2, dsize=(0, 0), fx=0.5, fy=0.5)
                    #adicionar no array
                    list_array_temp.append(im1_s)
                    contador += 1

                    #quando o resto do total de imagens for 0
                    #imagina uma matriz 3x3 onde y vai ser o total de imagens
                    if contador == 3 and len(list_array_temp) == 3:
                        list_array.append(list_array_temp)
                        list_array_temp = []
                        contador = 0
                        controle = True
                        temp = False
                    #quando o resto do total de imagens for maior que zero
                    #imagina uma matriz 2x2 onde y vai ser o total de imagens
                    elif contador == 2 and (len(os.listdir(subdiretorio2)) % 3 > 0):
                        list_array.append(list_array_temp)
                        list_array_temp = []
                        contador = 0
                    #bug sempre vai ignorar a ultima imagem da lista

                    #controle = False  
                concatenacao = cv2.vconcat([cv2.hconcat(im_list_h) for im_list_h in list_array])
                cv2.imwrite(pathSub +'\\'+ 'final.jpg', concatenacao)
    

def getImages(link, pagina_base,diretorio):
    
    try:
        dados_pagina = requests.get(link, timeout=30)
    except:
        print('Erro ao abrir a página' + pagina)

    sopa = BeautifulSoup(dados_pagina.content,'lxml')

    images = sopa.find_all('img')
    links = set()

    for image in images:
        if ('src' in image.attrs):
            #url = str(image.get('src'))
            url = urljoin(pagina_base, str(image.get('src')))
            title = str(image.get('title')).lower()
            #print(title)
            if '.jpg' in url or '.jpeg' in url or '.png' in url:
                if url.__contains__('covid-19') or url.__contains__('coronavírus') or url.__contains__('coronavirus') or url.__contains__('covid')  or title.__contains__('covid-19') or title.__contains__('coronavírus') or title.__contains__('coronavirus') or title.__contains__('covid'):
                    print(url)
                    links.add(url)
                
    for a in links: 
        try:
            j = requests.get(a, timeout=30)
            hash = random.getrandbits(128)
            
            #Estudar forma e evoluir isso
            #remover_acentos = a.split('/')[-1].split('.')[0].lower()
            #removeSpecialChars = remover_acentos.translate ({ord(c): " " for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+áàãíìóòõç"})
            with open(diretorio+'\\Images\\'+str(hash)+'.png', 'wb') as f:
                f.write(j.content)
        except Exception as e:
            print(e)

def crawl(paginas, profundidade,diretorio):
    x = datetime.datetime.now()
    data = x.strftime("%Y/%m/%d")
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    user_agent = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    for i in range(profundidade):
        novas_paginas = set()
        for pagina in paginas:
            http = urllib3.PoolManager(10,headers=user_agent)
            try:
                dados_pagina = http.request('GET',pagina)
            except:
                print('Erro ao abrir a página' + pagina)
                continue
            
            sopa = BeautifulSoup(dados_pagina.data,'lxml')
            
            noticiaslinks = set()
            links = sopa.find_all('a')
            contador = 1
            for link in links:

                if ('title' in link.attrs):
                    title = str(link.get('title').lower())
                    url = urljoin(pagina, str(link.get('href')))

                    if title.__contains__('coronavirus') or title.__contains__('coronavírus') or title.__contains__('COVID-19') or title.__contains__('covid'):
                        if url[-1] == '/':
                            tamanho = len(url)
                            url = url[:tamanho-1]
                        noticiaslinks.add(url)

                if ('href' in link.attrs):
                    url = urljoin(pagina, str(link.get('href')))
                        
                    if url.find("'") != -1:
                        continue
                    url = url.split('#')[0]                    
                    if url[0:4] == 'http' or url[0:5] == 'https':
                        novas_paginas.add(url)
                        if url.__contains__('coronavirus') or url.__contains__('coronavírus') or url.__contains__('COVID-19') or url.__contains__('covid'):
                            if url[-1] == '/':
                                tamanho = len(url)
                                url = url[:tamanho-1]
                            noticiaslinks.add(url)
            
                    contador = contador + 1
            for link in noticiaslinks:
                gravarNoArquivoUrl(pagina,link,diretorio)
                getImages(link,pagina,diretorio)
                    
            print(contador)
            
if __name__ == '__main__':
    #Start
    #Pasta base
    path = '.\Jornais'
    diretorio = os.listdir(path)
    for a in diretorio:
        pathSub = path + '\\'+ a
        subdiretorio =  os.listdir(pathSub)
        for file in subdiretorio:
            if 'txt' in file:
                fullpath = pathSub+'\\'+file
                sites = LerArquivo(fullpath)
                print(fullpath)
                crawl(sites,1,pathSub)
    gerarImagensConcatenadas()

In [ ]:
#Base conhecimento inicio
#import cv2
#import numpy as np
#img1 = cv2.imread('./Images/alcool.png')
#im1 = cv2.resize(img1, (200, 200)) 
#img2 = cv2.imread('./Images/alcool.png')
#im2 = cv2.resize(img2, (200, 200))
#img3 = cv2.imread('./Images/alcool.png')
#im3 = cv2.resize(img3, (200, 200))
#vis = np.concatenate((im1), axis=1)
#cv2.imshow('image', vis)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
#cv2.imwrite('out.png', vis)